# Updating a proofreading table or a list id file

This notebook contains functions and example scripts to update a proofreading table or lists of ids stored in an excel or txt file

Content per section:
<br>A) Updating IDs and presynaptic counts
<br>B) Updating main postsynaptic neurons database (table)
<br>B2) Updating all postsynaptic neurons databases (loop)
<br>C) Updating a list of IDs from a txt file
<br>D) Updating a list of IDs from any excel file
<br>E) Making a list of unique values

In [2]:
import os
import pandas as pd
import numpy as np
import fafbseg
import math
import itertools
from fafbseg import flywire
from caveclient import CAVEclient
from helpers.synapse_queries import combine_xyz, separate_xyz, synapse_count, filter_points, calculate_distance
client = CAVEclient('flywire_fafb_production')

C:\Users\sebas\anaconda3\envs\EM\lib\site-packages\blessed\terminal.py:186: UserWarning: Failed to setupterm(kind='xterm-color'): Could not find terminal xterm-color
  warnings.warn(msg)


## A) Updating IDs and presynaptic counts

### 1. Loading X neuron input neuron table

In [2]:
# Choose path and file
PC_disc = 'D'
dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets'
fileName = f'Tm3_neurons_input_count_R_20240610.xlsx'
filePath = os.path.join(dataPath,fileName)

In [3]:
#Loading file as DataFrame
df = pd.read_excel(filePath)
df.head()

,presynaptic_ID,counts,postsynaptic_ID,symbol,guess,lab,author,identified_in,seg_id,Updated_presynaptic_ID,Update_confidence_pre,Updated_counts,duplicates,notes,Working on,optic_lobe_id,hemisphere,Updated_postsynaptic_ID,Update_confidence_post,dorso-ventral
0,asdf,NaN,asdf,NaN,NaN,NaN,NaN,NaN,asdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,720575940630785730,39.0,720575940631441464,L1,Lamina monopolar 1; L1,NaN,96,NaN,720575940630785730,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
2,720575940623092044,21.0,720575940631441464,L1,Lamina monopolar 1; L1,NaN,2630,NaN,720575940623092044,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
3,720575940604687532,21.0,720575940631441464,Mi1,Mi1,NaN,100,NaN,720575940604687532,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
4,720575940624334276,21.0,720575940631441464,Dm1,Dm1; Distal medullary amacrine neuron 1; FBbt_...,NaN,392,NaN,720575940624334276,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN


In [4]:
#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["postsynaptic_ID"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)

#df = df[df['counts'] >= 3].copy() # Removing rows based on absolute count filter
    
#Getting the lists of IDs to update
segmentIDs = df["seg_id"].copy()
pre_IDs = df["presynaptic_ID"].copy()
post_IDs = df["postsynaptic_ID"].copy()
df.head()


,presynaptic_ID,counts,postsynaptic_ID,symbol,guess,lab,author,identified_in,seg_id,Updated_presynaptic_ID,Update_confidence_pre,Updated_counts,duplicates,notes,Working on,optic_lobe_id,hemisphere,Updated_postsynaptic_ID,Update_confidence_post,dorso-ventral
0,720575940630785730,39.0,720575940631441464,L1,Lamina monopolar 1; L1,NaN,96,NaN,720575940630785730,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
1,720575940623092044,21.0,720575940631441464,L1,Lamina monopolar 1; L1,NaN,2630,NaN,720575940623092044,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
2,720575940604687532,21.0,720575940631441464,Mi1,Mi1,NaN,100,NaN,720575940604687532,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
3,720575940624334276,21.0,720575940631441464,Dm1,Dm1; Distal medullary amacrine neuron 1; FBbt_...,NaN,392,NaN,720575940624334276,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN
4,720575940623928525,19.0,720575940631441464,L5,Lamina monopolar 5; L5,NaN,96,NaN,720575940623928525,NaN,NaN,NaN,NaN,NaN,NaN,Tm3-R31,R,NaN,NaN,NaN


In [5]:
#Replacing the 'INPUTS PROOFREAD' labelled row of the lists for the previous postsynaptic cell ID
for i, id in enumerate(pre_IDs):
    if id == 'INPUTS PROOFREAD':
        segmentIDs[i] = post_IDs[i-1] # The previous postsynaptic cell ID
        pre_IDs[i] = post_IDs[i-1]
        post_IDs[i] = post_IDs[i-1]

### 2. Updating IDs considering our excel file extructure (fast and new version)

In [6]:
#Updating segments (trying to update in loops to not collapse the server)

confidence_of_update_pre = []
confidence_of_update_post = []
updated_presynaptic_ID_column = []
updated_postsynaptic_ID_column = []
pre_ID_i = 0
post_ID_i = 0
rounds_of = 100
curr_round = 0
print(f'Total rounds to perform: {math.ceil((len(pre_IDs)/rounds_of))}')

for i in range(0,math.ceil((len(pre_IDs)/rounds_of))):
    curr_round += 1
    
    #pre_IDs
    curr_pre_IDs = pre_IDs[pre_ID_i:pre_ID_i+rounds_of]
    temp_segmentIDs_df = flywire.update_ids(curr_pre_IDs.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_presynaptic_ID_column.append(temp_segmentIDs_df["new_id"].tolist())
    confidence_of_update_pre.append(temp_segmentIDs_df["confidence"].tolist())
    pre_ID_i +=rounds_of
    
    #post_IDs
    curr_post_IDs = post_IDs[post_ID_i:post_ID_i+rounds_of]
    temp_segmentIDs_df = flywire.update_ids(curr_post_IDs.tolist(), stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_postsynaptic_ID_column.append(temp_segmentIDs_df["new_id"].tolist())
    confidence_of_update_post.append(temp_segmentIDs_df["confidence"].tolist())
    post_ID_i +=rounds_of
    
    if curr_round%5 == 0: #printing current round every 5 rounds
        print(curr_round)

updated_presynaptic_ID_column = list(itertools.chain.from_iterable(updated_presynaptic_ID_column))
updated_postsynaptic_ID_column = list(itertools.chain.from_iterable(updated_postsynaptic_ID_column))
confidence_of_update_pre = list(itertools.chain.from_iterable(confidence_of_update_pre))
confidence_of_update_post = list(itertools.chain.from_iterable(confidence_of_update_post))

Total rounds to perform: 705


Updating:   0%|          | 0/100 [00:00<?, ?it/s]

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

...

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

Updating:   0%|          | 0/100 [00:00<?, ?it/s]

705


### 2. Updating IDs considering our excel file extructure (old and slow version) - DO NOT USE -

Updating segments will be slow since we do it in a loop. Please just run this part if you really consider that the IDs in which your are mostly interested in need an update

In [ ]:
#Updating segments 
updated_presynaptic_ID_column = []
confidence_of_update = []
count = 1
for id in pre_IDs:
    count =+ 1
    if id == 'INPUTS PROOFREAD':
        updated_presynaptic_ID_column.append('INPUTS PROOFREAD')
        confidence_of_update.append('INPUTS PROOFREAD')
    else:
        temp_segmentIDs_df = flywire.update_ids(id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
        updated_presynaptic_ID_column.append(str(temp_segmentIDs_df["new_id"][0]))
        confidence_of_update.append(temp_segmentIDs_df["confidence"][0])
    print(f'row: {count} {updated_presynaptic_ID_column[-1]}')
    

### 2.1 Adding important columns for next steps

In [7]:
### Setting important columns to str
df["presynaptic_ID"] = df["presynaptic_ID"].astype(str)
df["postsynaptic_ID"] = df["postsynaptic_ID"].astype(str)
df["seg_id"] = df["seg_id"].astype(str)

In [8]:
### Updating the dataframe
# Adding the new url column to the data frame
df["Updated_presynaptic_ID"] = updated_presynaptic_ID_column
df["Updated_presynaptic_ID"] = df["Updated_presynaptic_ID"].astype(str) 
df["Updated_postsynaptic_ID"] = updated_postsynaptic_ID_column
df["Updated_postsynaptic_ID"] = df["Updated_postsynaptic_ID"].astype(str) 
df["Update_confidence_pre"] = confidence_of_update_pre
df["Update_confidence_post"] = confidence_of_update_post

In [9]:
len(confidence_of_update_pre)

70401

### 3. Updating counts between pre- and post synaptic partners

Strategy to save time:
1. (Step1) Create a dictionary with postsynaptic neuron's IDs as KEYS and their a input-neuron dataframe as VALUES , and then (step2) create a loop across presynaptic IDs to get the exact counting from the input-neuron-dataframe of postsynaptic neurons, loading the correct input-neuron-dataframe from the dictionary each time.

    Or, all in one single step: start a loop across unique postsynaptic IDs (be careful that the order is in the same as in the excel file, print them), get the input-neuron dataframe per each one in each round of the loop, and add a second loop across presynaptic IDs to get the exact counting.
    

2. Save the new countings in order of production in the same dataframe, as well as a column showing duplicated rows

In [10]:
#Implementing the "single step" option

unique_post_IDs = df[df['presynaptic_ID'] != 'INPUTS PROOFREAD']['postsynaptic_ID'].unique()
new_count_ls =  [] # For storing the new counts
copy_count_ls = [] # For storing the copy counts 
for post_id in unique_post_IDs: # loop across postsynaptic ids.
    if flywire.is_latest_root([post_id])[0]:
        curr_id = post_id
        print(f'Getting inputs from: {curr_id}')
    else:
        updated_ID_df = flywire.update_ids(post_id, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
        curr_id = updated_ID_df["new_id"][0]
        print(f'Getting inputs from: {curr_id} (updated)')
    
    curr_df = df[df['postsynaptic_ID'] == post_id].copy()
    curr_neurons_inputs = flywire.synapses.fetch_synapses(curr_id, pre=False, post=True, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "live")
    
    if curr_neurons_inputs.empty: #Adding this to fix isues with retrieveing data
        #Adding NaNs
        empty_list = [None]* len(df[df['postsynaptic_ID'] == post_id])
        new_count_ls = new_count_ls + empty_list
        copy_count_ls = copy_count_ls + empty_list
    
    else: 
        
        #Filtering redundant / duplicated counts (threshold = 100 nm)
        combine_xyz(curr_neurons_inputs)
        points = curr_neurons_inputs['pre_pt_position'].tolist()
        points_no_duplicates = filter_points(points, threshold_distance = 100)
        curr_neurons_inputs_no_duplicates = curr_neurons_inputs[curr_neurons_inputs['pre_pt_position'].apply(lambda x: x in points_no_duplicates)].copy()

        pre_id_copies_dict = {} # For checking ID duplicates, triplicates, etc
        for i,pre_id in enumerate(curr_df["Updated_presynaptic_ID"]): # loop across presynaptic ids
            #Counting copies
            if pre_id in pre_id_copies_dict.keys():
                pre_id_copies_dict[pre_id]+= 1 # duplicates, triplicates ...
            else:
                pre_id_copies_dict[pre_id] = 1 # initial count

            c = len(curr_neurons_inputs_no_duplicates[curr_neurons_inputs_no_duplicates['pre_pt_root_id'] == int(pre_id)])
            new_count_ls.append(c) # count between specific pre and post
            copy_count_ls.append(pre_id_copies_dict[pre_id])
            #print(f'Counts with {pre_id}: {c}, confidence {curr_df["Update_confidence"].tolist()[i]}, copy {pre_id_copies_dict[pre_id]}')

        #In old files with this rows acting as separators
        #new_count_ls.append('INPUTS PROOFREAD')
        #copy_count_ls.append('INPUTS PROOFREAD')
        


Getting inputs from: 720575940631441464
Getting inputs from: 720575940614981672
Getting inputs from: 720575940607340555
Getting inputs from: 720575940626335761
Getting inputs from: 720575940631316498
Getting inputs from: 720575940622067717
Getting inputs from: 720575940608430091
Getting inputs from: 720575940627302415
Getting inputs from: 720575940629028880
Getting inputs from: 720575940644306980
Getting inputs from: 720575940626616322
Getting inputs from: 720575940621639713
Getting inputs from: 720575940605917216
Getting inputs from: 720575940630237239
Getting inputs from: 720575940614096936
Getting inputs from: 720575940607365129
Getting inputs from: 720575940628393988
Getting inputs from: 720575940621936666
Getting inputs from: 720575940625096743
Getting inputs from: 720575940631685162
Getting inputs from: 720575940630196279
Getting inputs from: 720575940612190248
Getting inputs from: 720575940627963934
Getting inputs from: 720575940618311734
Getting inputs from: 720575940632379410


Getting inputs from: 720575940623804907
Getting inputs from: 720575940628302355
Getting inputs from: 720575940634356193
Getting inputs from: 720575940623211022
Getting inputs from: 720575940640892019 (updated)
Getting inputs from: 720575940633971169
Getting inputs from: 720575940647143988
Getting inputs from: 720575940633117265
Getting inputs from: 720575940626328126
Getting inputs from: 720575940621799960
Getting inputs from: 720575940620159573
Getting inputs from: 720575940631425611
Getting inputs from: 720575940639593038
Getting inputs from: 720575940629455415
Getting inputs from: 720575940639533646
Getting inputs from: 720575940627264030
Getting inputs from: 720575940626573886
Getting inputs from: 720575940621859354
Getting inputs from: 720575940632392263
Getting inputs from: 720575940621197900
Getting inputs from: 720575940644772388
Getting inputs from: 720575940629383754
Getting inputs from: 720575940627642938
Getting inputs from: 720575940632461881
Getting inputs from: 720575940

Getting inputs from: 720575940612019162
Getting inputs from: 720575940622560244
Getting inputs from: 720575940642268104
Getting inputs from: 720575940619502552
Getting inputs from: 720575940624241613
Getting inputs from: 720575940629867515
Getting inputs from: 720575940629947330
Getting inputs from: 720575940634487778
Getting inputs from: 720575940618161099
Getting inputs from: 720575940624067563
Getting inputs from: 720575940628159464
Getting inputs from: 720575940611435474
Getting inputs from: 720575940618228763
Getting inputs from: 720575940617550902
Getting inputs from: 720575940620878928
Getting inputs from: 720575940625650736
Getting inputs from: 720575940618277917
Getting inputs from: 720575940627782673
Getting inputs from: 720575940628411409
Getting inputs from: 720575940623496265
Getting inputs from: 720575940625601613
Getting inputs from: 720575940616203318
Getting inputs from: 720575940617829445
Getting inputs from: 720575940632077368
Getting inputs from: 720575940627080236


Getting inputs from: 720575940629626368
Getting inputs from: 720575940641437173
Getting inputs from: 720575940629671468
Getting inputs from: 720575940624557565
Getting inputs from: 720575940636966377
Getting inputs from: 720575940627775019
Getting inputs from: 720575940630234624
Getting inputs from: 720575940636343732
Getting inputs from: 720575940610541006
Getting inputs from: 720575940624797223
Getting inputs from: 720575940619888075
Getting inputs from: 720575940627711492
Getting inputs from: 720575940634944959
Getting inputs from: 720575940631496210
Getting inputs from: 720575940628956631
Getting inputs from: 720575940621665732
Getting inputs from: 720575940618358299
Getting inputs from: 720575940619122299
Getting inputs from: 720575940610170436
Getting inputs from: 720575940630341180
Getting inputs from: 720575940609445474
Getting inputs from: 720575940619691569
Getting inputs from: 720575940615308906
Getting inputs from: 720575940633966192
Getting inputs from: 720575940616742517


In [11]:
# Adding new columns to the main dataframe
df["Updated_counts"] = new_count_ls
df["duplicates"] =  

df.head()

,presynaptic_ID,counts,postsynaptic_ID,symbol,guess,lab,author,identified_in,seg_id,Updated_presynaptic_ID,Update_confidence_pre,Updated_counts,duplicates,notes,Working on,optic_lobe_id,hemisphere,Updated_postsynaptic_ID,Update_confidence_post,dorso-ventral
0,720575940630785730,39.0,720575940631441464,L1,Lamina monopolar 1; L1,NaN,96,NaN,720575940630785730,720575940630785730,1.0,27,1,NaN,NaN,Tm3-R31,R,720575940631441464,1.0,NaN
1,720575940623092044,21.0,720575940631441464,L1,Lamina monopolar 1; L1,NaN,2630,NaN,720575940623092044,720575940623092044,1.0,19,1,NaN,NaN,Tm3-R31,R,720575940631441464,1.0,NaN
2,720575940604687532,21.0,720575940631441464,Mi1,Mi1,NaN,100,NaN,720575940604687532,720575940604687532,1.0,18,1,NaN,NaN,Tm3-R31,R,720575940631441464,1.0,NaN
3,720575940624334276,21.0,720575940631441464,Dm1,Dm1; Distal medullary amacrine neuron 1; FBbt_...,NaN,392,NaN,720575940624334276,720575940624334276,1.0,16,1,NaN,NaN,Tm3-R31,R,720575940631441464,1.0,NaN
4,720575940623928525,19.0,720575940631441464,L5,Lamina monopolar 5; L5,NaN,96,NaN,720575940623928525,720575940623928525,1.0,15,1,NaN,NaN,Tm3-R31,R,720575940631441464,1.0,NaN


### 4. Saving back to the excell file

In [12]:
# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath)
writer = pd.ExcelWriter(filePath, engine = 'openpyxl')
writer.book = book

df.to_excel(writer, sheet_name='Updated_dataframe_'+date_str)
writer.save()
writer.close()

### 3. Or, saving in a new excel file

In [13]:
#Saving in a new file

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

file_name = f'Tm3_neurons_input_count_segments_update_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
df.to_excel(savePath, sheet_name='Segments update')

In [14]:
savePath

'D:\\Connectomics-Data\\FlyWire\\Excels\\drive-data-sets\\Tm3_neurons_input_count_segments_update_28Jun2024.xlsx'

### Faster update (code in progress...)

In [ ]:
#Updating segments (FAST)
#Do all segments as once, but filter out columns with 'INPUTS PROOFREAD' first
filtered_df = df[df["presynaptic_ID"] =! 'INPUTS PROOFREAD'].copy()
segmentIDs = filtered_df["seg_id"]
pre_IDs = filtered_df["presynaptic_ID"]
post_IDs = filtered_df["postsynaptic_ID"]

new_segmentIDs_df = flywire.update_ids(segmentIDs, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
new_segmentIDs = new_segmentIDs_df["new_id"].tolist()


In [ ]:
### Updating the dataframe
# Adding the new url column to the data frame
filtered_df["Updated_seg_id"] = new_segmentIDs
filtered_df["Updated_seg_id"] = filtered_df["Updated_seg_id"].astype(str) 

In [ ]:
# Code not finished....

# B) Updating main postsynaptic neurons database

### 1. Loading postsynaptic neuron dataframe

In [3]:
# Choose path and file
PC_disc = 'D'
dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\database'
fileName = f'Tm9 proofreadings.xlsx'
filePath = os.path.join(dataPath,fileName)

In [4]:
#Loading file as DataFrame
df = pd.read_excel(filePath)
if df["seg_id"][0] == 'asdf': #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)
df = df.astype(str)
display(df.head())
segmentIDs = df["Updated_seg_id"].tolist()

,XYZ-ME,symbol,XYZ-LO,quality comment,Mi4-connected,rim_area (Y/N),hemisphere,lab,author,name,...,optic_lobe_id,patch_id,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),dorso-ventral
0,"63464.0,83140.0,5499.0",Tm9,"82124.0,73675.0,5135.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L1,V,V1,Y,N,N,nan,N,?,V
1,"67273.0,85576.0,5497.0",Tm9,"84894.0,75635.0,5033.0",nan,nan,?,L,Marion Silies,Jonas Chojetzki,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L2,V,V2,Y,Y,Y,?,Y,Y,V
2,"70063.0,88589.0,5760.0",Tm9,"86449.0,78034.0,4816.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L3,V,V3,Y,Y,Y,Y,N,Y,V
3,"65296.0,86194.0,6202.0",Tm9,"81554.0,74784.0,4819.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L4,V,V4,N,N,N,nan,N,?,V
4,"69273.0,86122.0,4826.0",Tm9,"88273.0,76603.0,5126.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L5,V,V5,Y,N,N,nan,N,?,V


In [5]:
print(segmentIDs[0:5])

['720575940631867211', '720575940629938298', '720575940619904728', '720575940626539592', '720575940616954181']


### 2. Update with CAVE (not preferred)

In [ ]:
#Update IDs witth chunkedgraph module of CAVE

# For "segmentsIDs"
#Empty spaces are type float and will be filled with "0"
segmentsIDs_int = list(map(lambda x: 0 if type(x) == float else int(x),segmentIDs)) # From str to int
#to create a np.zeros array is important for the next step
new_segmentsIDs_int = list(map(lambda x: np.zeros(1) if x == 0 else client.chunkedgraph.get_latest_roots(x),segmentsIDs_int))
#Updated IDs leading to more than one ID a single ID will be kept inside [] brakets.
new_segmentsIDs_str = list(map(lambda x: str(x[0]) if x.size == 1 else x,new_segmentsIDs_int)) # From int to str

In [ ]:
# For "pre_IDs"
#Empty spaces are type float and will be filled with "0"
pre_IDs_int = list(map(lambda x: 0 if type(x) == float or x == 'INPUTS PROOFREAD'  else int(x),pre_IDs)) # From str to int
#to create a np.zeros array is important for the next step
new_pre_IDs_int = list(map(lambda x: np.zeros(1) if x == 0 else client.chunkedgraph.get_latest_roots(x),pre_IDs_int))
#Updated IDs leading to more than one ID a single ID will be kept inside [] brakets.
new_pre_IDs_str = list(map(lambda x: str(x[0]) if x.size == 1 else x,new_pre_IDs_int)) # From int to str

In [ ]:
# For "post_IDs"
#Empty spaces are type float and will be filled with "0"
post_IDs_int = list(map(lambda x: 0 if type(x) == float else int(x),post_IDs)) # From str to int
#to create a np.zeros array is important for the next step
new_post_IDs_int = list(map(lambda x: np.zeros(1) if x == 0 else client.chunkedgraph.get_latest_roots(x),post_IDs_int))
#Updated IDs leading to more than one ID a single ID will be kept inside [] brakets.
new_post_IDs_str = list(map(lambda x: str(x[0]) if x.size == 1 else x,new_post_IDs_int)) # From int to str

In [ ]:
### Seleting the right pre_ID if the update gaves more than one
#Getting the correct pre_IDs than contact each post_ID

from functools import reduce

correct_IDs = {}
curr_post_ID = 'Start'
for idx,pre_IDs in  enumerate(new_pre_IDs_str):
        
    #If there are multiple IDs in an array
    if type(pre_IDs) != str and type(pre_IDs) == np.ndarray:
        #Creatting synapses dataframe only once per each post_ID
        if curr_post_ID != new_post_IDs_str[idx]:
            synapses = flywire.synapses.fetch_synapses(new_post_IDs_str[idx], pre=False, post=True, attach=True,
                                          min_score=50, clean=True, transmitters=False,
                                          neuropils=True, batch_size=30,
                                          dataset='production', progress=True,mat="live")
            #Update post_ID
            print(f"Looking at post_ID: {new_post_IDs_str[idx]}") 
            curr_post_ID = new_post_IDs_str[idx]
            
        #Proof connectivity to the respective post_ID for each of them
        for ID in pre_IDs:
            if synapses[synapses['pre'] == ID].empty:
                continue
            else: # Only add the pre_ID (and its index) which has valid synapses with the post_IDs
                if idx in correct_IDs.keys(): # If there is already a valid ID, add other valid IDs 
                    curr_value =correct_IDs[idx]
                    new_value = curr_value+"_"+str(ID)
                    correct_IDs[idx] =new_value
                else:
                    correct_IDs[idx]=str(ID)

# Fixing the updated pre_IDs_str_list
for key, value in correct_IDs.items():
    new_pre_IDs_str[key] = value
    


In [ ]:
### Updating the dataframe
# Adding the new url column to the data frame
df["Updated_pre_IDs"] = new_pre_IDs_str
df["Updated_post_IDs"] = new_post_IDs_str

### 2. Update with FAFB (predered)

In [6]:
# Updating the segmentIDs
new_segmentIDs_df = flywire.update_ids(segmentIDs, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

Updating:   0%|          | 0/1481 [00:00<?, ?it/s]

In [7]:
new_segmentIDs_df["confidence"].unique()

array([1.  , 0.97, 0.95, 0.99, 0.75, 0.89])

In [8]:
### Updating the dataframe
# Adding the new column to the data frame
df["Update_confidence"] = new_segmentIDs_df["confidence"].astype(str).tolist()
df["Updated_seg_id"] = new_segmentIDs_df["new_id"].astype(str).tolist()
df.head()

,XYZ-ME,symbol,XYZ-LO,quality comment,Mi4-connected,rim_area (Y/N),hemisphere,lab,author,name,...,optic_lobe_id,patch_id,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),dorso-ventral
0,"63464.0,83140.0,5499.0",Tm9,"82124.0,73675.0,5135.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L1,V,V1,Y,N,N,nan,N,?,V
1,"67273.0,85576.0,5497.0",Tm9,"84894.0,75635.0,5033.0",nan,nan,?,L,Marion Silies,Jonas Chojetzki,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L2,V,V2,Y,Y,Y,?,Y,Y,V
2,"70063.0,88589.0,5760.0",Tm9,"86449.0,78034.0,4816.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L3,V,V3,Y,Y,Y,Y,N,Y,V
3,"65296.0,86194.0,6202.0",Tm9,"81554.0,74784.0,4819.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L4,V,V4,N,N,N,nan,N,?,V
4,"69273.0,86122.0,4826.0",Tm9,"88273.0,76603.0,5126.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L5,V,V5,Y,N,N,nan,N,?,V


In [9]:
#Checking duplicates in ids
df[df["Updated_seg_id"].duplicated() == True]

,XYZ-ME,symbol,XYZ-LO,quality comment,Mi4-connected,rim_area (Y/N),hemisphere,lab,author,name,...,optic_lobe_id,patch_id,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),dorso-ventral


In [10]:
#Checking updated ids
df[df["Update_confidence"].astype(float) < 1.]

,XYZ-ME,symbol,XYZ-LO,quality comment,Mi4-connected,rim_area (Y/N),hemisphere,lab,author,name,...,optic_lobe_id,patch_id,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),dorso-ventral
37,"54698.0,57524.0,5800.0",Tm9,"75082.0,56266.0,5383.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L38,D,D11,Y,N,N,nan,Y,N,D
308,"63273.0,85176.0,6032.0",Tm9,"81113.0,74569.0,4947.0",nan,nan,?,L,Borst Lab,Amalia Braun,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L309,nan,nan,N,N,N,nan,N,?,V
497,"75858.0,64623.0,3445.0",Tm9,"90216.0,63152.0,6096.0",nan,nan,?,L,Marion Silies,Jonas Chojetzki,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L498,nan,nan,Y,N,N,nan,N,?,V
538,"75954.0,86210.0,4327.0",Tm9,"92845.0,77280.0,5202.0",nan,nan,?,L,Axel Borst Lab,Amalia Braun,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L539,nan,nan,Y,N,N,nan,N,?,V
623,"54845.0,66831.0,5547.0",Tm9,"76772.0,62359.0,5406.0",nan,nan,?,L,Marion Silies,Lena Lörsch,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,L624,nan,nan,Y,N,Y,nan,Y,N,D
1252,"192857.0,95078.0,4876.0",Tm9,"173998.0,83272.0,3753.0",strange dendrite,nan,Y,R,Eyewire,Krzysztof Kruk,Tm9; Transmedullary neuron 9; FBbt_00003797,...,R562,nan,nan,N,N,N,nan,Y,N,V
1386,"180784.0,96416.0,4089.0",Tm9,"166805.0,85596.0,4004.0",strange dendrite,nan,Y,R,nan,nan,nan,...,R696,nan,nan,N,N,N,nan,?,N,nan


### 3. If wished: Reorder rows based on condition

In [ ]:
sorted_df = df.sort_values(by = 'cluster_id').copy()

In [39]:
sorted_df = df.copy()

In [57]:
df["seg_id"]

0       720575940631867211
1       720575940629938298
2       720575940619904728
3       720575940626539592
4       720575940616954181
               ...        
1370    720575940602880736
1371    720575940612861462
1372    720575940620517312
1373    720575940615223199
1374    720575940619077686
Name: seg_id, Length: 1375, dtype: int64

### 3. If wished: Add specific column status based on another file

### 3.1 Based on a txt with list of ids

In [13]:
# Selecting column for the update and file from which the info comes
column_to_update = 'rim_area (Y/N)' # 'inputs_proofread (Y/N)', detached_lamina (Y/N), 'healthy_L3 (Y/N)', 'rim_area (Y/N)'

update_file_path = r'D:\Connectomics-Data\FlyWire\Txts\cell_type_proofread'
update_file_with = 'root_ids_Tm9_outer_rim_20231018.txt' # list of ids

update_filePath =os.path.join(update_file_path,update_file_with)

# extractring info from the specific file
update_file_with_df = pd.read_csv(update_filePath)
update_file_with_ids_list = update_file_with_df.columns.tolist()

# Updating the list 
df[f'{column_to_update}_updated'] = np.where(df['seg_id'].isin(update_file_with_ids_list), "Y", "N")
df.head()

,XYZ-ME,symbol,XYZ-LO,quality comment,Mi4-connected,rim_area (Y/N),hemisphere,lab,author,name,...,column_id,backbone proofread (Y/N),twigs proofread (Y/N),inputs_proofread (Y/N),dark_neurons (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),dorso-ventral,rim_area (Y/N))_updated,rim_area (Y/N)_updated
0,"63464.0,83140.0,5499.0",Tm9,"82124.0,73675.0,5135.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,V1,Y,N,N,nan,N,?,V,N,N
1,"67273.0,85576.0,5497.0",Tm9,"84894.0,75635.0,5033.0",nan,nan,?,L,Marion Silies,Jonas Chojetzki,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,V2,Y,Y,Y,?,Y,Y,V,N,N
2,"70063.0,88589.0,5760.0",Tm9,"86449.0,78034.0,4816.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,V3,Y,Y,Y,Y,N,Y,V,N,N
3,"65296.0,86194.0,6202.0",Tm9,"81554.0,74784.0,4819.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,V4,N,N,N,nan,N,?,V,N,N
4,"69273.0,86122.0,4826.0",Tm9,"88273.0,76603.0,5126.0",nan,nan,?,L,Marion Silies,Sebastian Molina-Obando,"Tm9, Tm9_R, transmedullary neuron Tm9, FBbt_00...",...,V5,Y,N,N,nan,N,?,V,N,N


### 3.2 Or, Based on excel files with ids in a column and extra information in other columns

In [41]:
# Selecting column for the update and file from which the info comes
#For XYZ columns
update_file_path = r'E:\Connectomics-Data\FlyWire\Excels'
update_file_with = f'XYZ_df_{date}.xlsx'
update_filePath =os.path.join(update_file_path,update_file_with)

# extractring info from the specific file
update_file_with_df = pd.read_excel(update_filePath)
update_file_with_df.head()


,XYZ-ME,XYZ-LO,Updated_seg_id,seg_id
0,"63464.0,83140.0,5499.0","82124.0,73675.0,5135.0",720575940631867211,720575940631867211
1,"67273.0,85576.0,5497.0","84894.0,75635.0,5033.0",720575940629938298,720575940629938298
2,"70063.0,88589.0,5760.0","86449.0,78034.0,4816.0",720575940619904728,720575940619904728
3,"65296.0,86194.0,6202.0","81554.0,74784.0,4819.0",720575940626539592,720575940626539592
4,"69273.0,86122.0,4826.0","88273.0,76603.0,5126.0",720575940616954181,720575940616954181


In [42]:
#Transfering info from one dataframe to another based on a reference column (here seg_id)
def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

In [43]:
# Function inputs
source_cols = ['XYZ-ME', 'XYZ-LO','seg_id']
target_cols = ['XYZ-ME', 'XYZ-LO','seg_id']
reference_column = 'seg_id'

# Selecting dataframes and resetting index
source_df = update_file_with_df[source_cols].copy()
source_df.reset_index(inplace = True, drop = True)
target_df = df[target_cols].copy()
target_df.reset_index(inplace = True, drop = True)


source_df = source_df.astype(str)
target_df = target_df.astype(str)

# Running the function and compleating the dataset
result_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_df.head()

,XYZ-ME,XYZ-LO,seg_id
0,"63464.0,83140.0,5499.0","82124.0,73675.0,5135.0",720575940631867211
1,"67273.0,85576.0,5497.0","84894.0,75635.0,5033.0",720575940629938298
2,"70063.0,88589.0,5760.0","86449.0,78034.0,4816.0",720575940619904728
3,"65296.0,86194.0,6202.0","81554.0,74784.0,4819.0",720575940626539592
4,"69273.0,86122.0,4826.0","88273.0,76603.0,5126.0",720575940616954181


In [44]:
# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath)
writer = pd.ExcelWriter(filePath, engine = 'openpyxl')
writer.book = book

result_df = result_df.astype(str)
result_df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

### 3. If wished: Add the center of mass of postsynaptic sites

In [15]:
#Function
def find_center_point(points, threshold):
    if isinstance(points, list):
        points = np.array(points)

    # Calculate the distances between each point and all other points
    distances = np.linalg.norm(points[:, np.newaxis] - points, axis=2)

    # Calculate the average distance for each point
    avg_distances = np.mean(distances, axis=1)

    # Find the indices of points within the threshold distance
    valid_indices = np.where(avg_distances < threshold)[0]

    # Check if there are any valid points
    if len(valid_indices) > 0:
        # Calculate the geometric center of valid points
        center_point = np.mean(points[valid_indices], axis=0)
        #Rounding
        center_point = np.round(center_point, decimals=1)
        
        # Find the closest point to the center
        closest_point_index = np.argmin(np.linalg.norm(points[valid_indices] - center_point, axis=1))
        closest_point = points[valid_indices][closest_point_index]
    else:
        center_point = np.array([0,0,0])
        closest_point = np.array([0,0,0])

    return center_point.tolist(), closest_point.tolist()

In [16]:
def combine_xyz(df):
    """
    Combines separated x, y and z column into one, changes units and adds new column names for
    generating a neuroglancer link with function nglui.statebuilder.helpers.make_synapse_neuroglancer_link

    Args:
        pandas data frame containing x,y and z as columns of the same length

    Returns:
        same pandas data frame containing a new column with [x/4,y/4,z/40] lists
    """
    # Generating the single column

    post_pt_position = []
    for x,y,z in zip(df['post_x'].tolist(),df['post_y'].tolist(),df['post_z'].tolist()):
        temp_ls = [x/4,y/4,z/40]
        post_pt_position.append(temp_ls)

    pre_pt_position = []
    for x,y,z in zip(df['pre_x'].tolist(),df['pre_y'].tolist(),df['pre_z'].tolist()):
        temp_ls = [x/4,y/4,z/40]
        pre_pt_position.append(temp_ls)

    #Adding new columns and names
    df['post_pt_position'] = post_pt_position
    df['pre_pt_position'] = pre_pt_position
    #Changing column names
    df.rename(columns={'pre': 'pre_pt_root_id', 'post': 'post_pt_root_id'}, inplace=True)

In [17]:
#Loading postsynaptic coordinated for each neuron in a specific neuropile and getting the center point
#The loop can take quite a lot of time depending on the ammount of ids!


#Looping for several rounds of X (e.g, 100)
rounds_of = 100
loop_number = 1 # Must start at "1" if not run before
start_point = (rounds_of*loop_number) - rounds_of
 
#Shortening the df to priorize id:

#short_df = df[(df['detached_lamina (Y/N)'] == 'N') &(df['inputs_proofread (Y/N)'] == 'Y')].copy()
short_df = df[df['XYZ-ME'] == 'nan'].copy()

In [18]:
len(short_df)

24

In [19]:
round(len(short_df)/rounds_of)

0

In [21]:
# Looping across chosen rows

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

for i in range(loop_number, round(len(short_df)/rounds_of)+2):
    print(f'Loop #: {loop_number}')
    curr_df = short_df[start_point:start_point+rounds_of].copy()
    curr_df.reset_index(inplace=True,drop=True)

    XYZ_ME = []
    XYZ_LO = []
    for seg_id in curr_df["Updated_seg_id"]:

        #Print
        print(f'currently at: {seg_id}')
        # Getting coordinates of synaptic contacts
        neurons_inputs = flywire.synapses.fetch_synapses(seg_id, pre=False, post=True, attach=True, 
                                                     min_score=50, clean=True, transmitters=False, 
                                                     neuropils=True, batch_size=30, 
                                                     dataset='production', progress=True,mat= "live")
        combine_xyz(neurons_inputs)
        threshold = 5000

        try:
            #Find the center point with medulla coordinates
            neurons_inputs_ME = neurons_inputs[neurons_inputs['neuropil'].str.contains('ME')].copy()
            points = neurons_inputs_ME['post_pt_position'].tolist()
            center_point, closest_point = find_center_point(points, threshold)
            XYZ_ME.append(closest_point)
        except: # to catch some point clouds that have no ME 
            XYZ_ME.append([0,0,0])
            
        try:
            #Find the center point with lobula coordinates
            neurons_inputs_LO = neurons_inputs[neurons_inputs['neuropil'].str.contains('LO')].copy()
            points = neurons_inputs_LO['post_pt_position'].tolist()
            center_point, closest_point = find_center_point(points, threshold)
            XYZ_LO.append(closest_point)
        except: # to catch some point clouds that have no LO labels
            XYZ_LO.append([0,0,0])
            

    XYZ_LO_strings = [','.join(map(str, sublist)) for sublist in XYZ_LO]
    XYZ_ME_strings = [','.join(map(str, sublist)) for sublist in XYZ_ME]

    #saving
    XYZ_df = pd.DataFrame(XYZ_ME_strings, columns=['XYZ-ME'])
    XYZ_df['XYZ-LO'] = XYZ_LO_strings
    XYZ_df['Updated_seg_id'] =  curr_df['Updated_seg_id']
    XYZ_df['seg_id'] =  curr_df['seg_id']
    XYZ_df.to_excel(f'D:\Connectomics-Data\FlyWire\Excels\drive-data-sets\XYZ_df_{loop_number}_{date_str}.xlsx', index=False)
    start_point += rounds_of
    loop_number += 1


Loop #: 1
currently at: 720575940627437443
currently at: 720575940631341100
currently at: 720575940628746664
currently at: 720575940638219120
currently at: 720575940619127124
currently at: 720575940638124265
currently at: 720575940634654945
currently at: 720575940632399119
currently at: 720575940619126868
currently at: 720575940614403043
currently at: 720575940618919837
currently at: 720575940619127892
currently at: 720575940638187120
currently at: 720575940615083048
currently at: 720575940629702699
currently at: 720575940623843085
currently at: 720575940616973836
currently at: 720575940615231697
currently at: 720575940619128148
currently at: 720575940627221239
currently at: 720575940628377363
currently at: 720575940628837909
currently at: 720575940623214583
currently at: 720575940629697502


In [ ]:

## Testing function with some manually inserted dots
import numpy as np
import matplotlib.pyplot as plt

# Example XYZ coordinates
points = [
    [66659, 85714, 5406],
    [66885, 85749, 5367],
    [67124, 86454, 5390],
    [67664, 86251, 5434],
    [67273, 85576, 5497],
    [66399, 85482, 5428],
    [66803, 85360, 5477],
    [66505, 85552, 5413],
    [66772, 85838, 5450],
    [66496, 85540, 5402],
    [72452, 82402, 5694],
    [75373, 80809, 5808]
]

points = neurons_inputs_LO['post_pt_position'].tolist()

# Convert the points to a NumPy array
points = np.array(points)

# Set the threshold distance to exclude outliers
threshold = 5000

# Find the center point
center_point, closest_point  = find_center_point(points, threshold)

if center_point is not None:
    print("Center Point:", center_point)
else:
    print("No valid points within the threshold.")

# Plotting the points and center point
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot for the points
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='y', label='Points')

# Scatter plot for the center point
if center_point is not None:
    ax.scatter(center_point[0], center_point[1], center_point[2], c='r', marker='x', label='Center Point')
    ax.scatter(closest_point [0], closest_point [1], closest_point [2], c='b', marker='x', label='closest_point')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()

plt.show()

### 4. Saving back to excel file

In [23]:

# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath)
writer = pd.ExcelWriter(filePath, engine = 'openpyxl')
writer.book = book

df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

# B2) Updating all postsynaptic neurons databases (loop)

### 1. Loading data bases of interest in a loop 

In [ ]:
# Data paths
# Choose path and file
PC_disc = 'E'
dataPath = f'{PC_disc}:\Connectomics-Data\FlyWire\Excels\drive-data-sets\database'

fileName_ls = glob(dataPath +"\\"+ "*.xlsx")


#Creating the database in a loop
df_ls = []
for fileName in fileName_ls:
    print(f'Importing: {fileName}')
    filePath = os.path.join(dataPath,fileName)
    df = pd.read_excel(filePath)
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    if df["seg_id"][0] == 'asdf': 
        df = df.iloc[1: , :]
        df.reset_index(inplace=True,drop=True)
    df_ls.append(df)

# C) Updating of list of IDs from a txt file / or a plain excell file


### 1. Loading the data from a txt file

In [95]:
# Choose path and file
dataPath = r'D:\Connectomics-Data\FlyWire\Txts\cell_type_proofread'
fileDate = '20231106'
fileName = f'root_ids_T4d_R_{fileDate}.txt'
#fileName = f'temp.txt'
filePath = os.path.join(dataPath,fileName)
ids_df = pd.read_csv(filePath, sep = ",")
curr_ID_ls = ids_df.columns.tolist()
curr_ID_ls = [s for s in curr_ID_ls if ".1" not in s]
print(curr_ID_ls)

['720575940615263298', '720575940622343548', '720575940603498357', '720575940604436720', '720575940604679664', '720575940605181152', '720575940610345426', '720575940612549490', '720575940613289443', '720575940615524806', '720575940615758130', '720575940618657403', '720575940619944363', '720575940620553069', '720575940620894630', '720575940621380566', '720575940622232402', '720575940624804178', '720575940626277434', '720575940627024069', '720575940627230750', '720575940627756968', '720575940628559403', '720575940628864247', '720575940630444396', '720575940631561784', '720575940631830083', '720575940632861267', '720575940633071827', '720575940633665049', '720575940635931243', '720575940636022903', '720575940640632528', '720575940646143267', '720575940647837572', '720575940649653113', '720575940600934665', '720575940603356896', '720575940603556981', '720575940603842476', '720575940604567486', '720575940604637045', '720575940604652512', '720575940604699825', '720575940604738737', '72057594

In [96]:
#Updating the IDs via Fafbseg
updated_ID_df = fafbseg.flywire.update_ids(curr_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

Updating:   0%|          | 0/778 [00:00<?, ?it/s]

In [97]:
updated_ID_df[updated_ID_df['confidence'] < 0.8]#['new_id'].tolist()

,old_id,new_id,confidence,changed
415,720575940625923203,720575940629758825,0.56,True
474,720575940627579529,720575940636603640,0.79,True


In [98]:
len(updated_ID_df['new_id'].tolist())
len(updated_ID_df['new_id'].unique().tolist())


777

### 2. Saving data in txt file

In [99]:
#Saving the data
updated_fileName = f'Updated_{fileName}'
updated_filePath = os.path.join(dataPath,updated_fileName)

id_list = list(set(updated_ID_df['new_id'].unique().tolist()))
with open(updated_filePath , "w") as output:
    output.write(str(id_list))


### 1. Or, loading the data from an excell file

In [4]:
# Choose path and file
PC_disc = 'D'
dataPath = f'{PC_disc}:\FlyWire-Data\database'
date = '20231017'
fileName = f'All_IDs_anatomical_RF_analysis.xlsx'
filePath = os.path.join(dataPath,fileName)

#Loading file as DataFrame
df = pd.read_excel(filePath)


#Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
if df["seg_id"][0] == 'asdf': 
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)

#Dropping dupllicates
result_df = df.drop_duplicates(subset=["Updated_seg_id"], keep='first').copy()

#Quick look on the dataframe
display(result_df.head())
    
#Getting the lists of IDs to update
curr_ID_ls = result_df["Updated_seg_id"].tolist()

,symbol,hemisphere,seg_id,Updated_seg_id,optic_lobe_id,backbone proofread (Y/N)
0,Tm9,R,720575940615761570,720575940632916783,R1,Y
1,Tm9,R,720575940630668871,720575940628906795,R2,Y
2,Tm9,R,720575940627187849,720575940639049038,R3,Y
3,Tm9,R,720575940633362131,720575940641658676,R4,Y
4,Tm9,R,720575940612657522,720575940638132058,R5,Y


In [5]:
#Updating all IDs at once
updated_ID_df = fafbseg.flywire.update_ids(curr_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

Updating:   0%|          | 0/5188 [00:00<?, ?it/s]

In [ ]:
# Or, Updating the IDs via Fafbseg in a for loop

_start = 0
_steps = 100
_last = _steps
_rounds =  round(len(curr_ID_ls)/ _steps) +1

print(f'Rounds to perform: {_rounds}')
updated_ID_df = pd.DataFrame()
for i in range(0,_rounds):
    #print(f'Round {i}')
    curr_ID_df = fafbseg.flywire.update_ids(curr_ID_ls[_start:_last], stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
    updated_ID_df = pd.concat([updated_ID_df,curr_ID_df])
    _start = _start + _steps
    _last = _last + _steps


In [6]:
result_df.reset_index(drop=True, inplace=True)
result_df['Updated_seg_ids'] = updated_ID_df['new_id'].astype(str).tolist()
result_df['Updated_confidence'] = updated_ID_df['confidence'].astype(str).tolist()

In [7]:
result_df.tail()

,symbol,hemisphere,seg_id,Updated_seg_id,optic_lobe_id,backbone proofread (Y/N),Updated_seg_ids,Updated_confidence
5183,TmY17,R,720575940633505934,720575940633505934,TmY17-R152,Y,720575940633505934,1.0
5184,TmY17,R,720575940651270902,720575940651270902,TmY17-R153,Y,720575940651270902,1.0
5185,TmY17,R,720575940625696637,720575940625696637,TmY17-R154,Y,720575940625696637,1.0
5186,TmY17,R,720575940625313428,720575940625313428,TmY17-R155,Y,720575940625313428,1.0
5187,TmY17,R,720575940613612394,720575940613612394,TmY17-R156,Y,720575940613612394,1.0


In [8]:
display(updated_ID_df.head())
print('Update confidences: ')
print(set(updated_ID_df['confidence'].tolist()))

,old_id,new_id,confidence,changed
0,720575940632916783,720575940632916783,1.0,False
1,720575940628906795,720575940628906795,1.0,False
2,720575940639049038,720575940639049038,1.0,False
3,720575940641658676,720575940641658676,1.0,False
4,720575940638132058,720575940638132058,1.0,False


Update confidences: 
{0.98, 1.0, 0.89, 0.99, 0.95, 0.97, 0.76, 0.92, 0.93, 0.79, 0.91, 0.77, 0.85}


In [9]:
updated_ID_df[updated_ID_df['confidence'] < 1]

,old_id,new_id,confidence,changed
416,720575940616417653,720575940608436914,0.98,True
690,720575940626719365,720575940615203880,0.99,True
817,720575940629161713,720575940624355583,0.95,True
990,720575940638014064,720575940622113633,0.99,True
1062,720575940631729103,720575940625951207,0.89,True
1065,720575940617200601,720575940620195707,0.97,True
1373,720575940623910349,720575940651036281,0.98,True
1458,720575940624885685,720575940632539407,0.99,True
1550,720575940624218549,720575940645836580,0.99,True
1609,720575940635789943,720575940632612138,0.97,True


### 2. Saving back in the excel file

In [10]:
# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath)
writer = pd.ExcelWriter(filePath, engine = 'openpyxl')
writer.book = book

result_df.to_excel(writer, sheet_name='Updated_table_'+date_str) #sorted_df
writer.save()
writer.close()

In [102]:
updated_ID_df[updated_ID_df['new_id'] == 720575940613560342]

,old_id,new_id,confidence,changed


### E) Making a list of unique values

In [8]:
PC_disc = 'D'
dataPath =  f'{PC_disc}:\Connectomics-Data\FlyWire\Txts'

#File one
fileName_1 = f'temp.txt'
filePath = os.path.join(dataPath,fileName_1)
df_1 = pd.read_csv(filePath, delimiter = "\t",header=None)
list1 = list(set(df_1[0].tolist()))

# Saving function
def save_list_to_file(file_path, input_list):
    df = pd.DataFrame(input_list, columns=['Items'])
    df.to_csv(file_path, header=False, index=False)
    
    
PC_disc = 'D'

file_path_1 = os.path.join(dataPath,f'Unique_{fileName_1}')
save_list_to_file(file_path_1, list1)